In [1]:
import os
from itertools import product
import pandas as pd
import multiprocessing as mp

In [2]:
import subprocess

## IMPORTANT: Make sure that "Uniandes_Framework" is in .gitignore
framework_path = "Uniandes_Framework"

if os.path.exists(framework_path):
    # Pull updates if the framework is already cloned
    try:
        subprocess.run(["git", "-C", framework_path, "pull"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while pulling updates from the framework: {e}")
else:
    # Clone the framework if it is not already cloned
    try:
        subprocess.run(["git", "clone", "git@github.com:Phenomenology-group-uniandes/Uniandes_Framework.git"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while cloning the framework: {e}")
from Uniandes_Framework.delphes_reader.lhereader import LHE_Loader, readLHEF, get_event_by_child
from Uniandes_Framework.delphes_reader.root_analysis import get_kinematics_row, make_histograms, overlap_histos

hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.


Already up to date.
Welcome to JupyROOT 6.22/06


In [3]:
case = "woRHC_UpZp"
sim_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "SIMULATIONS", f"lq_zp_{case}")
signals = ["ta_ta","zp_tau_tau"]
xs_path = { signal: os.path.join( os.path.dirname(os.getcwd()), "01_signal_production", f"xs_signals_{case}", signal, "XS_Matrix.csv") for signal in signals}
# xs_path = { signal: 1 for signal in signals}
M_U= [1000]
G_U = [1.5]

csv_sim_path = pd.read_csv(os.path.join("Uniandes_Framework", "SimulationsPaths.csv"))
df = pd.concat(
    [csv_sim_path]
    +
    [ pd.DataFrame.from_dict(
    {
        "name" : signal + f"_m{m}_g{g}", 
        "path": os.path.join(sim_path, signal,f'M{m}_gU{g:.4f}'.replace('.', '_')),  
        'xs(pb)': 1.0
    }, 
    orient = "index").T for signal, m , g in product(signals,M_U,G_U) ]
)
# save the new csv
csv_sim_path = os.path.join(os.getcwd(), "SimulationsPaths.csv")
df.to_csv(csv_sim_path, index = False)


In [4]:
def get_kinematics(run):
    run_name = os.path.basename(os.path.dirname(run))
    childs=readLHEF(run)
    if os.path.isfile(run):
        print('Reading', run_name)
    else:
        pass
    
    results = list()
    for child in childs:
        event = get_event_by_child(child)
        taus = event.getParticlesByIDs([-15,15])
        if len(taus) == 2:
            taus[0].SetName("#tau_{1}")
            taus[1].SetName("#tau_{2}")
            sum_taus = taus[0].TLV + taus[1].TLV
        else :
            continue
        row = get_kinematics_row(taus)
        row["m_ll"] = sum_taus.M()
        results.append(row)
    print(run_name, 'done!')
    return results


In [5]:
# Load the simulation

def load_simulation(x):
    signal, m, g = x
    lhe_loader = LHE_Loader(signal + f"_m{m}_g{g}", csv_sim_path)
    results = list()
    
    n_cores = 3
    with mp.Pool(n_cores) as pool:
        for result in list(pool.map(get_kinematics, lhe_loader.Forest)):
            results += result
    return signal, pd.DataFrame.from_records(results)

In [6]:
a = dict(map(load_simulation, product(signals,M_U,G_U)))

ta_ta_m1000_g1.5 imported with 15 trees!
/home/Cristian/SIMULATIONS/lq_zp_woRHC_UpZp/ta_ta/M1000_gU1_5000
Reading run_01
Reading run_09
Reading run_05
run_01 done!
run_09 done!
run_05 done!
Reading run_02
Reading run_06
Reading run_10
run_02 done!
run_06 done!
run_10 done!
Reading run_03
Reading run_07
Reading run_11
run_03 done!
run_07 done!
run_11 done!
Reading run_04
Reading run_08
Reading run_12
run_04 done!
run_08 done!
run_12 done!
Reading run_13
run_13 done!
Reading run_14
run_14 done!
Reading run_15
run_15 done!


ParseError: mismatched tag: line 575743, column 2 (<string>)

In [ ]:
a['ta_ta'].head()

In [ ]:
default_hist_bins_dict={
    "#Delta{R}":[96,0,7],
    "#Delta{#eta}":[80,-5,5],
    "#Delta{#phi}":[52,-3.25,3.25],
    "#Delta{pT}":[120, 0.0, 1500.0],
    "#Delta{#vec{pT}}":[240, 0.0, 4800.0],
    "#Delta{#vec{p}}":[240, 0.0, 4800.0],
    "MET(GeV)":[80, 0.0, 1000.0],
    "pT_": [160, 0.0, 2000.0],
    "sT(GeV)": [200, 0.0, 4000.0],
    "mT(GeV)": [200, 0.0, 4000.0],
    "#eta_":[80, -5, 5],
    "#phi_":[128, -3.2, 3.2],
    "Energy_":[80, 0.0, 1000.0],
    "m_ll" : [160, 0.0, 2000.0]
}
root_histos ={ signal: make_histograms(a[signal],hist_bins_dict = default_hist_bins_dict) for signal in signals}

In [ ]:
root_histos

In [ ]:
histos, canvas, legend = overlap_histos( 'pT_{#tau_{1}}(GeV)', root_histos, log_scale =True)

In [ ]:
histos, canvas, legend = overlap_histos( 'pT_{#tau_{2}}(GeV)', root_histos, log_scale =True)

In [ ]:
histos, canvas, legend = overlap_histos('#Delta{R}_{#tau_{1}#tau_{2}}', root_histos, log_scale =True)

In [ ]:
histos, canvas, legend = overlap_histos('m_ll', root_histos, log_scale =True)
canvas.SaveAs("mll.pdf")